In [1]:
import numpy as np
import pandas as pd
import statistics as stats
import quantstats as qs
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as wb
import plotly.express as px
import warnings 
warnings.filterwarnings('ignore')

## Dataframe a partir de 2023-01-01

In [2]:
df = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/orders_results_20_07.csv")
df.sort_values('user_id')
# df = df.loc['2023-04-01':]

,date,user_id,s10i_login,platform_id,instance_id,strategy_id,category,number_of_traded_stocks,result,normalized_result
175043,2023-05-22,132,mateuslana,1000,3535182,224,multibroker,38,4.00,4.000000
223794,2023-05-18,132,mateuslana,1000,3535400,1176,multibroker,122,-2380.01,-858.364262
223796,2023-05-19,132,mateuslana,1000,3535400,1176,multibroker,166,990.00,280.301205
223795,2023-05-22,132,mateuslana,1000,3535400,1176,multibroker,156,2472.00,744.769231
223790,2023-05-15,132,mateuslana,1000,3535400,1176,multibroker,280,-5082.00,-943.800000
...,...,...,...,...,...,...,...,...,...,...
106703,2023-07-19,781173,Lopes1,1000,3592138,1065,multibroker,200,-24.00,-0.240000
76582,2023-07-19,781173,Lopes1,1000,3592127,1063,multibroker,200,-30.00,-0.300000
130183,2023-07-18,781210,GuilhermeAmorim,1000,3591940,1058,multibroker,4,80.00,80.000000
129266,2023-07-19,781210,GuilhermeAmorim,3,3592656,1060,monobroker,2,74.00,74.000000


In [3]:
# clients_iat_list = [754204,760657,753994,729346,683556,770583,771265,584518,3838,771239,770357,743407,116574,723167,690113,499861,771241]
# df = df[df['user_id'].isin(clients_iat_list)]
df_user = df.copy()

# Agrupamento por usuário (s10i_login)

In [4]:
users_list = df['user_id'].unique().tolist()
strategies_per_user = []
days_per_robot = []
for user in users_list:
    df_test = df_user.copy()
    df_test=df_test[df_test['user_id'] ==user]
    n_strategies = np.where(df_test['strategy_id'].duplicated(),0,1).sum()
    n_instances = np.where(df_test['instance_id'].duplicated(),0,1).sum()
    robots_list = df_test['instance_id'].unique().tolist()
    for n in robots_list:
        maior_valor = None
        df_test_robots = df_test.copy()
        df_test_robots=df_test_robots[df_test_robots['instance_id'] == n]
        if maior_valor is None or len(df_test_robots) > maior_valor:
            maior_valor = len(df_test_robots)   

    pair_est = (user, n_strategies, n_instances, len(df_test)/n_instances, maior_valor)
    strategies_per_user.append(pair_est)

In [5]:
df_robots_and_est = pd.DataFrame(strategies_per_user, columns = ['user_id','number_of_strategies','number_of_robots','Mean_instances_dailys','max_dailys_in_robot'])
df_user_2 = df_user.copy()
df_user = df_user.groupby('user_id').sum()
df_merged = pd.merge(df_user, df_robots_and_est, on='user_id')

In [6]:
df_user_2

,date,user_id,s10i_login,platform_id,instance_id,strategy_id,category,number_of_traded_stocks,result,normalized_result
0,2023-02-13,108594,ALEXGANHA,1000,945332,216,multibroker,28,-50.00,-19.642857
1,2023-01-31,320358,marcusmoura,5900,1791140,588,smarttinvest,800,350.10,6.126750
2,2023-05-02,320358,marcusmoura,5900,1791140,588,smarttinvest,463,-213.20,-5.525702
3,2023-06-05,320358,marcusmoura,5900,1791140,588,smarttinvest,3135,2085.64,23.949933
4,2023-01-09,320358,marcusmoura,5900,1791140,588,smarttinvest,529,-74.65,-1.834499
...,...,...,...,...,...,...,...,...,...,...
280162,2023-07-19,388532,leofariasf,308,3593196,1156,monobroker,2,-280.00,-280.000000
280163,2023-07-19,157108,Boytrader,308,3593513,1058,monobroker,4,75.00,75.000000
280164,2023-07-19,115041,edilar,308,3593889,413,monobroker,4,30.00,30.000000
280165,2023-07-19,640418,eproszek,1000,3593906,518,multibroker,2,NaN,NaN


## Resultados Absolutos

### Usuários com maior Lucro Absoluto no período

In [7]:
df_user_2 = df_user_2.groupby('s10i_login').sum()

In [8]:
df_user_abs = df_user_2.copy()
df_user_abs.drop(['platform_id','instance_id','strategy_id','number_of_traded_stocks','normalized_result','user_id'], axis=1,inplace=True)
df_user_abs.sort_values('result').tail(20)

,result
s10i_login,
Cris01,55012.01
henriquegalante,56545.64
sandropires,57698.85
jorgeh01,57748.79
Glaubern,58518.99
bessjba,60425.00
Valdemy,60577.64
Figaalex,69147.90
47986395168,69450.00


In [9]:
df_user_abs_merged = df_merged.copy()

In [10]:
df_user_abs_pos = df_user_abs_merged[df_user_abs_merged['result']>0]
round(df_user_abs_pos['number_of_robots'].mean(),1)

14.9

In [11]:
df_user_abs_pos = df_user_abs_merged[df_user_abs_merged['result']>0]
round(df_user_abs_pos['number_of_strategies'].mean(),1)

2.4

In [12]:
df_user_abs_merged.sort_values('result').tail(10)

,user_id,platform_id,instance_id,strategy_id,number_of_traded_stocks,result,normalized_result,number_of_strategies,number_of_robots,Mean_instances_dailys,max_dailys_in_robot
2623,769615,314000,1115304698,67824,6442,78084.99,53470.808095,1,35,8.971429,10
259,73873,147476,1949281353,130134,54596,87026.00,37935.145038,2,29,19.379310,17
793,442030,3630810,13734677111,1234288,120674,97574.12,79015.446221,5,86,47.325581,30
1270,658438,2534640,3516996315,216432,33546,99082.99,97817.905272,1,251,3.992032,6
75,14895,1077000,3464968942,379790,92978,101854.00,15928.989876,6,29,37.137931,33
2404,763969,515000,1813190249,436782,18516,115028.03,81492.666846,2,49,10.510204,6
1888,742164,181000,612904461,165773,12810,123889.00,4591.506667,4,7,25.857143,25
1364,679137,831000,2862662364,708146,418640,141261.09,26061.294383,12,267,3.112360,2
1106,632488,198000,689528715,127701,338750,155488.57,8541.702483,2,46,4.304348,11
764,416434,2749360,4059178292,778204,76824,169780.79,72881.975944,10,43,27.720930,20


### Usuários com maior PERDA Absoluta no período

In [13]:
df_user_abs.sort_values('result').head(20)

,result
s10i_login,
claudineyleu,-279652.56
AAAArthur,-162165.00
asalgbr,-95500.00
laba25,-78665.00
03859189921,-56807.79
vfquadros,-55267.00
dudu.tsn,-50348.00
neo_diogomarques@yahoo.com.br,-45191.49
BuyHodl,-41903.74


In [14]:
df_user_abs_pos = df_user_abs_merged[df_user_abs_merged['result']<0]
round(df_user_abs_pos['number_of_robots'].mean(),1)

10.1

In [15]:
df_user_abs_pos = df_user_abs_merged[df_user_abs_merged['result']<0]
round(df_user_abs_pos['number_of_strategies'].mean(),1)

2.3

In [16]:
df_user_abs_merged.sort_values('result').head(10)

,user_id,platform_id,instance_id,strategy_id,number_of_traded_stocks,result,normalized_result,number_of_strategies,number_of_robots,Mean_instances_dailys,max_dailys_in_robot
641,320037,2358000,8148405868,1033785,364004,-279652.56,17624.828457,9,168,14.035714,14
1287,660059,10000,28558413,4130,2515,-162165.00,-3494.636232,1,3,3.333333,1
1774,735986,207,232504684,47679,1795,-95500.00,-6215.625000,1,11,6.272727,8
824,458141,1203000,4199449158,556600,50000,-78665.00,11481.449862,5,76,15.828947,15
1098,631207,57231,2478376192,180581,620166,-56807.79,1205.756476,4,84,8.440476,1
527,226299,248000,867340356,53568,294041,-55267.00,-9410.031944,1,20,12.400000,8
2729,772191,57232,212408412,17168,1528,-50348.00,-1372.550000,2,14,4.285714,4
320,104577,103000,311936561,23072,5938,-45191.49,34.319289,1,6,17.166667,20
1271,658701,43170,1506027766,376023,25532,-41903.74,72.981926,7,18,24.000000,4
1065,617739,100000,339367474,80279,3049,-39090.00,-2758.180000,5,8,12.500000,11


## Resultados Normalizados

In [17]:
df_user_norm = df_merged.copy().round(1)
df_user_norm.drop(['platform_id','instance_id','strategy_id','number_of_traded_stocks'], axis=1,inplace=True)
df_user_norm['normalized_result'] = np.where(df_user_norm['normalized_result']>df_user_norm['result'],df_user_norm['result'],df_user_norm['normalized_result'])
df_user_norm['normalized_per_robots'] = round(df_user_norm['normalized_result'] / df_user_norm['number_of_strategies'],1)

df_user_norm.sort_values('normalized_result').tail(20)

,user_id,result,normalized_result,number_of_strategies,number_of_robots,Mean_instances_dailys,max_dailys_in_robot,normalized_per_robots
203,47170,28601.0,21735.7,1,39,9.5,12,21735.7
248,66652,24164.1,24164.1,1,19,20.7,7,24164.1
131,22624,55012.0,24784.9,2,22,7.8,5,12392.4
1049,606876,31368.0,25416.3,1,13,17.1,10,25416.3
1274,659045,25537.0,25537.0,14,71,17.5,20,1824.1
650,328125,25992.0,25862.8,1,4,77.2,85,25862.8
1364,679137,141261.1,26061.3,12,267,3.1,2,2171.8
1320,664919,69450.0,27583.8,8,688,2.0,1,3448.0
205,47455,31608.8,28201.4,6,37,4.8,1,4700.2
439,168191,69147.9,33937.3,2,158,3.9,7,16968.6


In [18]:
df_user_norm_test = df_merged.copy().round(1)
df_user_norm_test.drop(['platform_id','instance_id','strategy_id','normalized_result'], axis=1,inplace=True)
df_user_norm_test['normalized_result'] = df_user_norm_test['result'] / (df_user_norm_test['number_of_traded_stocks']/2)
# df_user_norm_test['normalized_result'] = np.where(df_user_norm_test['normalized_result']>df_user_norm_test['result'],df_user_norm_test['result'],df_user_norm_test['normalized_result'])
df_user_norm_test['normalized_per_robots'] = round(df_user_norm_test['normalized_result'] / df_user_norm_test['number_of_robots'],1)

df_user_norm_test.sort_values('normalized_per_robots').tail(20)

,user_id,number_of_traded_stocks,result,number_of_strategies,number_of_robots,Mean_instances_dailys,max_dailys_in_robot,normalized_result,normalized_per_robots
1835,738301,69,1803.0,1,1,9.0,9,52.260870,52.3
2817,774257,4,118.0,1,1,2.0,2,59.000000,59.0
2697,771296,4,119.0,1,1,1.0,1,59.500000,59.5
2509,766106,10,301.0,1,1,5.0,5,60.200000,60.2
1449,693632,8,247.0,1,1,2.0,2,61.750000,61.8
1585,721082,26,1857.0,1,2,2.0,2,142.846154,71.4
2379,763757,2,74.0,1,1,1.0,1,74.000000,74.0
1221,651590,2,76.0,1,1,1.0,1,76.000000,76.0
1138,637297,26,1053.0,1,1,14.0,14,81.000000,81.0
1797,737111,8,340.0,1,1,1.0,1,85.000000,85.0


# Agrupamento por Estratégia (strategy_id)

In [19]:
df_strategy_list = df.copy().round(1)
df_strategy_list


,date,user_id,s10i_login,platform_id,instance_id,strategy_id,category,number_of_traded_stocks,result,normalized_result
0,2023-02-13,108594,ALEXGANHA,1000,945332,216,multibroker,28,-50.0,-19.6
1,2023-01-31,320358,marcusmoura,5900,1791140,588,smarttinvest,800,350.1,6.1
2,2023-05-02,320358,marcusmoura,5900,1791140,588,smarttinvest,463,-213.2,-5.5
3,2023-06-05,320358,marcusmoura,5900,1791140,588,smarttinvest,3135,2085.6,23.9
4,2023-01-09,320358,marcusmoura,5900,1791140,588,smarttinvest,529,-74.6,-1.8
...,...,...,...,...,...,...,...,...,...,...
280162,2023-07-19,388532,leofariasf,308,3593196,1156,monobroker,2,-280.0,-280.0
280163,2023-07-19,157108,Boytrader,308,3593513,1058,monobroker,4,75.0,75.0
280164,2023-07-19,115041,edilar,308,3593889,413,monobroker,4,30.0,30.0
280165,2023-07-19,640418,eproszek,1000,3593906,518,multibroker,2,NaN,NaN


### Estratégias mais lucrativas de forma absoluta no período

In [20]:
df_strategy = df_strategy_list.groupby('strategy_id').sum()

In [21]:
df_strategy

,user_id,platform_id,instance_id,number_of_traded_stocks,result,normalized_result
strategy_id,,,,,,
216,4448099380,12491544,37304530060,936878,634333.9,518735.5
224,41333020217,78228203,316199098481,11365780,-79720.4,285945.2
293,16680754,10176,129706971,656,-730.0,-284.9
295,233495278,81352,1255443411,14581,-589.0,1384.2
317,2117044241,4251778,12450756253,30579,35697.0,9504.1
...,...,...,...,...,...,...
1296,975992,3000,10759024,30,-141.0,-28.2
1305,122418,1000,3590075,6,0.0,0.0
1307,4606795,10000,35880925,64,434.0,217.0


In [22]:
df_strategy_abs = df_strategy.copy().round(1)
df_strategy_abs.drop(['platform_id','user_id','instance_id','number_of_traded_stocks','normalized_result'], axis=1,inplace=True)


df_strategy_abs.sort_values('result').tail(20)

,result
strategy_id,
1099,18350.0
1185,18450.0
901,27778.0
1103,34633.0
902,35530.0
317,35697.0
799,36471.8
1137,43008.0
962,51035.0


### Estratégias mais lucrativas de forma absoluta

In [23]:
df_strategy_abs.sort_values('result').head(20)

,result
strategy_id,
1176,-503086.6
413,-371335.2
588,-128836.2
1031,-106634.4
712,-96070.0
1126,-90492.0
385,-81344.0
224,-79720.4
1156,-75962.0


### Percentual de clientes com lucro por estratégia

In [24]:
lista_est = df['strategy_id'].unique().tolist()
pair_values = []
for i in lista_est:
    df_test = df[df['strategy_id']==i]
    df_test = df_test.groupby('s10i_login').sum()
    df_test['Positive'] = np.where(df_test['result'] > 0,1,0)
    value = round(df_test['Positive'].sum()/len(df_test),2)
    pair = (i, value, len(df_test))
    pair_values.append(pair)
    # print(f'Strategy ID {i} -- {value*100}%')
df_positive_est = pd.DataFrame(pair_values, columns = ['strategy_id','Positive_clients_%','Number_of_clients'])

In [25]:
df_positive_est.sort_values('Number_of_clients').tail(20)

,strategy_id,Positive_clients_%,Number_of_clients
31,905,0.20,83
18,857,0.51,86
56,1058,0.29,92
1,588,0.26,103
58,1060,0.45,107
29,417,0.33,112
20,1064,0.12,124
12,385,0.15,131
63,1156,0.33,154
23,913,0.40,162


In [26]:
len(df_positive_est)

208

In [27]:
lista_est = [1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1085,1095,1096,1097,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,935,936,937]
pair_values = []
for i in lista_est:
    df_test = df[df['strategy_id']==i]
    df_test = df_test.groupby('user_id').sum()
    df_test['Positive'] = np.where(df_test['result'] > 0,1,0)
    value = round(df_test['Positive'].sum()/len(df_test),2)
    pair = (i, value, len(df_test))
    pair_values.append(pair)
    # print(f'Strategy ID {i} -- {value*100}%')
df_positive_est = pd.DataFrame(pair_values, columns = ['strategy_id','Positive_clients_%','Number_of_clients'])

In [28]:
# df_positive_est.dropna(inplace=True)
df_positive_est.sort_values('Positive_clients_%')

,strategy_id,Positive_clients_%,Number_of_clients
26,927,0.00,1
19,920,0.00,11
18,919,0.00,1
6,1104,0.00,2
34,936,0.00,1
12,1110,0.00,4
35,937,0.06,18
9,1107,0.12,40
27,928,0.24,17
17,918,0.25,8


# TEste PASS

In [29]:
df_pass = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/pass_active.csv")
list_pass = df_pass['user_id'].unique().tolist()
lista_pass_id = [1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1085,1095,1096,1097,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,935,936,937]
lista_pass_id_own = [1085,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110]
lista_pass_id_partner = [918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,935,936,937]
lista_all_sync = [693,712,716,728,735,736,740,759,760,812,820,821,832,833,856,857,858,868,872,873,874,892,902,903,904,905,908,909,913,915,923,924,925,926,927,928,929,947,948,949,950,959,960,961,962,963,964,965,966,968,969,972,973,978,982,1115,1117,1118,1119,1121,1122,1123,1124,1125,1126,1137,1147,1148,1149,1150,1154,1157,1160,1166,1167]

In [30]:
df_pass_user = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/data_11_22.csv")

In [31]:
df_pass_user

,platform_id,user_id,instance_id,date,result,strategy_id,number_of_traded_stocks,normalized_result,category,s10i_login
0,308,558827,3047133,2022-11-01,-52.00,317,8,-26.000000,monobroker,Thiagoar
1,1000,664816,3023360,2022-11-01,73.00,224,10,73.000000,multibroker,flavio_fontana
2,1000,625332,3322276,2022-11-01,240.00,413,70,48.000000,multibroker,pipocasbt
3,1099,715580,3027299,2022-11-01,-437.00,224,18,-437.000000,inter,inter_smarttbot_7085883
4,308,659140,3323918,2022-11-01,-135.00,224,30,-45.000000,monobroker,Amortiis
...,...,...,...,...,...,...,...,...,...,...
305073,1000,403,3542342,2023-05-26,-206.00,413,170,-206.000000,multibroker,robertostreger
305074,1000,38101,3540510,2023-05-26,-272.00,1187,12,-136.000000,multibroker,dpaffrath
305075,1000,666033,3534479,2023-05-26,-309.99,1176,86,-72.090698,multibroker,sidmar
305076,4090,544900,3483744,2023-05-26,40.00,413,4,20.000000,monobroker,kharynyamauti


In [32]:
df_filtrado = df_pass_user[df_pass_user['strategy_id'].isin(lista_all_sync)]
# df_filtrado = df_filtrado[df_filtrado['user_id'].isin(list_pass)]

In [33]:
df_filtrado_test = df_filtrado.copy()

In [34]:
df_filtrado = df_filtrado.groupby('user_id').sum()
df_filtrado.drop(['platform_id','instance_id','strategy_id','number_of_traded_stocks','normalized_result'],axis=1,inplace=True)
df_filtrado['Positive']=np.where(df_filtrado['result']>0,1,0)
df_filtrado=df_filtrado.sort_values('result')

In [35]:
df_filtrado

,result,Positive
user_id,,
143097,-28268.0,0
649700,-21121.0,0
15212,-20432.0,0
7790,-20296.0,0
750315,-16798.0,0
...,...,...
438406,27954.0,1
442150,28647.0,1
748474,35245.0,1


In [36]:
df_filtrado.to_csv(r'C:\Users\IGOR NOGUEIRA BARROS\Downloads\df_filtrado_partner.csv', float_format='%.2f')

### Clientes com lucro no Pass

In [37]:
round(df_filtrado['Positive'].sum()/len(df_filtrado)*100,1)

37.0

In [38]:
valores_faltantes = list(set(list_pass) - set(df_filtrado.index))
valores_faltantes

[15104,
 229505,
 343818,
 419599,
 310547,
 400406,
 765338,
 746659,
 683556,
 755751,
 38568,
 416434,
 252851,
 734900,
 13890,
 766530,
 262340,
 736470,
 103398,
 765672,
 502125,
 419575,
 419581]

In [39]:
lista_all_pass_users = df_filtrado_test['user_id'].unique().tolist()
len(lista_all_pass_users)

768

In [40]:
df_filtrado_test

,platform_id,user_id,instance_id,date,result,strategy_id,number_of_traded_stocks,normalized_result,category,s10i_login
24,1000,742931,3304503,2022-11-01,201.0,858,30,26.800000,multibroker,Dorley
33,1000,266813,3066180,2022-11-01,26.0,740,6,21.666667,multibroker,sskred
52,1000,373226,3238355,2022-11-01,-379.0,857,20,-75.800000,multibroker,michelnsilva
56,1000,710724,3241154,2022-11-01,-160.0,857,8,-80.000000,multibroker,laercio1986
59,1000,565987,3336676,2022-11-01,-144.0,858,4,-72.000000,multibroker,Thamiris369
...,...,...,...,...,...,...,...,...,...,...
305054,1000,59356,3456022,2023-05-26,100.0,693,2,100.000000,multibroker,marcel.silva.k2
305059,1000,134417,3508054,2023-05-26,60.0,857,10,12.000000,multibroker,smarttrader
305061,1000,770650,3539143,2023-05-26,57.0,857,4,57.000000,multibroker,atila.mello
305063,1000,764808,3524051,2023-05-26,265.0,857,20,53.000000,multibroker,00806704152


In [41]:
df_test = pd.DataFrame()
clients_ids = []
for i in lista_all_pass_users:
    df_test = df_filtrado_test[df_filtrado_test['user_id'] == i]
    temp_ids = df_test['strategy_id'].unique().tolist()
    pair = (i, temp_ids)
    clients_ids.append(pair)

In [42]:
df_clients_ids = pd.DataFrame(clients_ids, columns = ['user_id','strategy_ids'])
df_clients_ids.to_csv(r'C:\Users\IGOR NOGUEIRA BARROS\Downloads\df_clients_ids.csv', float_format='%.2f')

In [43]:
df_pass_2 = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/results_by_date.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/IGOR NOGUEIRA BARROS/Downloads/results_by_date.csv'

In [ ]:
lista_est = [693,712,716,728,735,736,740,759,760,812,820,821,832,833,856,857,858,868,872,873,874,892,902,903,904,905,908,909,913,915,923,924,925,926,927,928,929,947,948,949,950,959,960,961,962,963,964,965,966,968,969,972,973,978,982,1115,1117,1118,1119,1121,1122,1123,1124,1125,1126,1137,1147,1148,1149,1150,1154,1157,1160,1166,1167]
pair_values = []
for i in lista_est:
    df_test = df_pass_2[df_pass_2['strategy_id']==i]
    df_test = df_test.groupby('user_id').sum()
    df_test['Positive'] = np.where(df_test['result'] > 0,1,0)
    value = round(df_test['Positive'].sum()/len(df_test),2)
    pair = (i, value, len(df_test))
    pair_values.append(pair)
    # print(f'Strategy ID {i} -- {value*100}%')
df_positive_est = pd.DataFrame(pair_values, columns = ['strategy_id','Positive_clients_%','Number_of_clients'])

In [ ]:
df_positive_est.dropna(inplace=True)
df_positive_est.sort_values('Positive_clients_%').tail(30)

,strategy_id,Positive_clients_%,Number_of_clients
20,874,0.43,14
58,1119,0.43,7
66,1147,0.46,13
22,902,0.48,25
65,1137,0.48,77
56,1117,0.50,2
53,978,0.50,4
47,965,0.50,2
0,693,0.50,2
18,872,0.50,2


In [ ]:
df_positive_est.to_csv(r'C:\Users\IGOR NOGUEIRA BARROS\Downloads\df_sync_ids.csv', float_format='%.2f')

In [ ]:
df_giro = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/results_by_date_21.csv")
# df_giro = df_giro[df_giro['strategy_id']==413]

In [ ]:
df_giro = df_giro.groupby('instance_id').sum()
df_giro.sort_values('result').tail(20)

,platform_id,user_id,result,strategy_id,number_of_traded_stocks,normalized_result
instance_id,,,,,,
3067183,23079,14644980,83011.22,16632,169,8.301122e+04
3392853,114000,47473476,89595.00,24624,4798,3.356985e+04
2946626,10990,6914780,90467.30,4130,658,4.523365e+04
3391947,67000,14870315,92169.00,57419,15900,1.319897e+03
3349412,88000,19531160,92174.00,75416,17230,1.671273e+03
2943221,1000,1561,99914.50,216,45,1.998290e+04
2950399,35420,72588805,100576.00,25760,160801,3.820244e+03
2818065,4090,630766,106239.99,413,55,2.124800e+04
2966171,85722,52739466,121346.26,17472,663,1.213463e+05


### Análise Clientes IAT

In [ ]:
clients_iat_list = [753994,729346,770583,3838,771239,770357,743407,723167,690113,499861,771241]

In [ ]:
754204,760657,753994,729346,683556,770583,771265,584518,3838,771239,770357,743407,116574,723167,690113,499861,771241

(754204,
 760657,
 753994,
 729346,
 683556,
 770583,
 771265,
 584518,
 3838,
 771239,
 770357,
 743407,
 116574,
 723167,
 690113,
 499861,
 771241)

In [ ]:
df_canais = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/test_216.csv")

In [ ]:
df_canais = df_canais[df_canais['strategy_id']==216]
df_canais.dropna(inplace=True)
df_canais.sort_values('result').tail(20)

,platform_id,user_id,instance_id,date,result,strategy_id,number_of_traded_stocks,normalized_result,category,s10i_login
3245,1000,540598,2520347,2021-08-03,2485.00,216,106,797.075472,multibroker,mvverde
13,1000,337065,2053028,2021-08-03,2582.00,216,50,51.640000,multibroker,traks1826
98,1000,32243,2491839,2021-08-03,2700.00,216,180,180.000000,multibroker,getulio.adm@gmail.com
2273,1000,2017,2435568,2021-08-03,2900.00,216,116,375.000000,multibroker,jaderjr
2007,1000,337065,2204796,2021-08-03,2915.00,216,50,58.300000,multibroker,traks1826
1679,1000,633877,2535733,2021-08-03,2920.00,216,370,268.324324,multibroker,LariPedrucci
2227,1000,14763,2168119,2021-08-03,3002.00,216,606,995.712871,multibroker,RafaelVarella
1846,1000,265184,2510507,2021-08-03,3200.00,216,38,673.684211,multibroker,ajmramos
2141,1000,174260,2522909,2021-08-03,3405.00,216,144,449.270833,multibroker,israel68
1749,1000,34035,2192277,2021-08-03,3580.00,216,10,358.000000,multibroker,AZ


In [ ]:
df_canais['result'].sum()

4178767.92

In [ ]:
df_canais = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/results_by_date_21.csv")

In [ ]:
df_canais.sort_values('date')

,platform_id,user_id,instance_id,date,result,strategy_id,number_of_traded_stocks,normalized_result,category,s10i_login
1315008,308,693127,3493326,2023-04-03,-132.0,413,12,-66.000,monobroker,Wulkytony
1316239,1000,218073,3309805,2023-04-03,-280.0,224,8,-70.000,multibroker,wilcruz
1316238,4090,416434,3338178,2023-04-03,140.0,800,4,140.000,monobroker,najao@gingamais.com.br
1316237,4090,766236,3461215,2023-04-03,140.0,800,4,140.000,monobroker,82382107049
1316236,1000,758771,3387897,2023-04-03,29.0,1064,400,0.145,multibroker,lpontesreis
...,...,...,...,...,...,...,...,...,...,...
1408072,1000,773182,3545645,2023-06-12,60.0,694,4,30.000,multibroker,Wrl
1408071,1000,719642,3520276,2023-06-12,40.0,857,20,4.000,multibroker,Marcioinada
1408070,1000,396101,3444401,2023-06-12,130.0,413,4,97.500,multibroker,Liberalino
1408068,1000,442030,3412930,2023-06-12,200.0,413,16,25.000,multibroker,sergiomoura


In [ ]:
df_canais = df_canais[df_canais['date'] >= '2023-04-01']

In [ ]:
df_canais

,platform_id,user_id,instance_id,date,result,strategy_id,number_of_traded_stocks,normalized_result,category,s10i_login
1315008,308,693127,3493326,2023-04-03,-132.0,413,12,-66.0,monobroker,Wulkytony
1315009,3,720612,3448489,2023-04-03,150.0,321,2,150.0,monobroker,00675317924
1315010,1000,34345,3413890,2023-04-03,529.0,1125,10,158.7,multibroker,ricamend
1315011,308,558827,3039042,2023-04-03,60.0,413,4,30.0,monobroker,Thiagoar
1315012,1000,631379,3477881,2023-04-03,144.0,224,16,72.0,multibroker,carpesjunior
...,...,...,...,...,...,...,...,...,...,...
1409221,1000,553457,3397742,2023-06-12,54.0,224,12,18.0,multibroker,90488270510
1409222,1000,749525,3494626,2023-06-12,-30.0,890,2,-30.0,multibroker,ivan2405
1409223,1000,373874,3557789,2023-06-12,-175.0,224,2,-175.0,multibroker,adalbertodelgado
1409224,3,766999,3508552,2023-06-12,150.0,224,10,30.0,monobroker,A@rtur


In [ ]:
list_pass = df_canais['user_id'].unique()

In [ ]:
list_pass

array([693127, 720612,  34345, ..., 763882, 368072,  22875], dtype=int64)

In [ ]:
len(list_pass)

1503

# Análise de clientes rodando o mesmo setup

In [ ]:
df_same_robots = pd.read_csv("C:/Users/IGOR NOGUEIRA BARROS/Downloads/same_robots.csv")

In [ ]:
users_list = df_same_robots['user_id'].unique().tolist()
duplicated_list = []
for user in users_list:
    df_aux = df_same_robots.copy()
    df_aux = df_aux[df_aux['user_id'] == user]
    is_duplicated = df_aux['params'].duplicated().sum()
    contagem_duplicacoes = df_aux['params'].value_counts().max()
    pair_duplicated = (user, is_duplicated, contagem_duplicacoes)
    duplicated_list.append(pair_duplicated)
df_duplicated = pd.DataFrame(duplicated_list, columns = ['user_id','Duplicados', 'Max'])

In [ ]:
df_duplicated=df_duplicated[df_duplicated['Duplicados'] >0]
df_duplicated.sort_values('Max').tail(15)

,user_id,Duplicados,Max
123,658401,2,3
48,11166,11,3
411,761182,2,3
358,411385,2,3
469,771023,2,3
21,719642,5,4
1,38101,3,4
149,763969,6,5
286,267113,4,5
390,774168,4,5


In [ ]:
df_duplicated.to_csv(r'C:\Users\IGOR NOGUEIRA BARROS\Downloads\df_duplicated.csv', float_format='%.2f')